#### Clean Data

In [1]:
#import packages
import pandas as pd
import math
import statsmodels.stats.api as sms
import scipy.stats as st
import numpy as np

In [11]:
#import data
raw_data = pd.read_csv("ab_data.csv")
df = raw_data.copy()

print("Number of rows: ", df.shape[0], " Number of columns: ", df.shape[1])
df

Number of rows:  286690  Number of columns:  6


,Unnamed: 0,user_id,timestamp,group,landing_page,converted
0,0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...,...
286685,294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
286686,294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
286687,294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
286688,294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [3]:
df["group"].value_counts()

treatment    143397
control      143293
Name: group, dtype: int64

In [4]:
#some of the control group saw the new_page and some tretment group saw the old_page - delete these instances
mask1 = (df["group"] == "control") & (df["landing_page"] == "new_page")
index_to_drop1 = df[mask1].index
df = df.drop(index_to_drop1)

mask2 = (df["group"] == "treatment") & (df["landing_page"] == "old_page")
index_to_drop2 = df[mask2].index
df = df.drop(index_to_drop2)

print(df.shape)
df["group"].value_counts()

(286690, 6)


treatment    143397
control      143293
Name: group, dtype: int64

In [5]:
#Check how many duplicated users exist
print(df["user_id"].count())
print(df["user_id"].nunique())

286690
286690


In [6]:
#drop duplicated users
df.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

In [21]:
#Show the % split between users who saw new vs old page
#Calculate pooled probability
mask = (df["group"] == "control")
conversions_control = df["converted"][mask].sum()
total_users_control = df["converted"][mask].count()

mask = (df["group"] == "treatment")
conversions_treatment = df["converted"][mask].sum()
total_users_treatment = df["converted"][mask].count()

print("División de usuarios del grupo de control que vieron la página antigua frente a usuarios del grupo de tratamiento que vieron la página nueva: ", 
          round(total_users_control / df["converted"].count() * 100, 2), "% ",
          round((total_users_treatment / df["converted"].count()) * 100, 2), "%")

print('\n--------------------------------------------------------------------------------------------------------------------------------------------------------\n')

#count number of users who converted in each group
print("Número de usuarios del grupo de control que se convirtieron al ver la página antigua: ", conversions_control)
print("Total de usuarios del grupo de control: ", total_users_control)
print("Porcentaje de usuarios del grupo de control que se convirtieron: ", round((conversions_control / total_users_control) * 100, 2), "%")

print('\n--------------------------------------------------------------------------------------------------------------------------------------------------------\n')

print("Número de usuarios del grupo de tratamiento que se convirtieron al ver la página nueva: ", conversions_treatment)
print("Total de usuarios del grupo de control: ", total_users_treatment)
print("Porcentaje de usuarios del grupo de tratamiento que se convirtieron: ", round((conversions_treatment/ total_users_treatment) * 100, 2), "%")

División de usuarios del grupo de control que vieron la página antigua frente a usuarios del grupo de tratamiento que vieron la página nueva:  49.98 %  50.02 %
Número de usuarios del grupo de control que se convirtieron al ver la página antigua:  17220
Total de usuarios del grupo de control:  143293
Porcentaje de usuarios del grupo de control que se convirtieron:  12.02 %
Número de usuarios del grupo de tratamiento que se convirtieron al ver la página nueva:  17025
Total de usuarios del grupo de control:  143397
Porcentaje de usuarios del grupo de tratamiento que se convirtieron:  11.87 %


In [8]:
df.head()
pv=pd.pivot_table(df,index='group',values='converted',aggfunc=[np.sum,np.mean])
pv

,sum,mean
,converted,converted
group,,
control,17220,0.120173
treatment,17025,0.118726


#### Set Test Parameters

In [23]:
#Check what sample size is required
baseline_rate = conversions_control / total_users_control
practical_significance = 0.01 #user defined
confidence_level = 0.05 #user defined, for a 95% confidence interval
sensitivity = 0.8 #user defined

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(
    effect_size = effect_size, 
    power = sensitivity, 
    alpha = confidence_level, 
    # se establece en 1 para indicar que se compara un grupo con otro grupo de tamaño igual
    ratio=1
) 

print("Required sample size: ", round(sample_size), " per group")

Required sample size:  17183  per group


### Tamaño del efecto 

El tamaño del efecto se refiere a la magnitud o fuerza de una diferencia o relación entre variables en un estudio. Es una medida estandarizada que indica la magnitud relativa de un efecto o la fuerza de una asociación en términos estadísticos. Un tamaño del efecto más grande indica una diferencia o relación más fuerte entre variables, mientras que un tamaño del efecto más pequeño indica una diferencia o relación más débil. El tamaño del efecto se utiliza para evaluar la importancia práctica o sustantiva de los resultados de un estudio y puede ser utilizado para comparar la magnitud de los efectos en diferentes estudios.

Es cierto que el tamaño del efecto es una medida utilizada para cuantificar la magnitud de la diferencia o relación entre dos grupos o variables. En el código que se muestra, se utiliza la función proportion_effectsize de la biblioteca statsmodels para calcular el tamaño del efecto en un estudio diseñado para comparar dos proporciones: una tasa de conversión en el grupo de control (conversions_control / total_users_control) y una tasa de conversión en otro grupo que se supone con una diferencia prácticamente significativa.

El parámetro practical_significance es una diferencia entre las tasas de conversión entre los grupos que se considera importante o relevante en un sentido práctico. El baseline_rate es la proporción o tasa de conversión en el grupo de control.

Al utilizar la función proportion_effectsize se está usando la diferencia entre el baseline_rate (el grupo de control) y la tasa de conversión que se considera prácticamente significativa (baseline_rate + practical_significance) como un parámetro para calcular el tamaño del efecto.

Este cálculo del tamaño del efecto se realiza para poder determinar el tamaño de muestra necesario para detectar una diferencia prácticamente significativa con una determinada probabilidad (dada por el nivel de confianza) y una potencia determinada por la sensibilidad (sensitivity).

### Tamaño de la muestra

El tamaño de la muestra, por otro lado, se refiere al número de observaciones o participantes en un estudio. Es la cantidad de datos que se recopilan o analizan en un estudio. El tamaño de la muestra es un factor importante en el diseño de estudios estadísticos, ya que afecta la precisión y confiabilidad de los resultados obtenidos. Un tamaño de muestra más grande tiende a proporcionar estimaciones más precisas y confiables de los parámetros poblacionales y aumenta la capacidad del estudio para detectar efectos o relaciones reales entre variables. Por lo tanto, el tamaño de la muestra es un aspecto importante en el diseño y planificación de estudios de investigación y análisis de potencia.

#### A/B Test

In [27]:
#Calculate pooled probability
mask = (df["group"] == "control")
conversions_control = df["converted"][mask].sum()
total_users_control = df["converted"][mask].count()

mask = (df["group"] == "treatment")
conversions_treatment = df["converted"][mask].sum()
total_users_treatment = df["converted"][mask].count()

prob_pooled = (conversions_control + conversions_treatment) / (total_users_control + total_users_treatment)

In [29]:
#Calculate pooled standard error and margin of error
se_pooled = math.sqrt(prob_pooled * (1 - prob_pooled) * (1 / total_users_control + 1 / total_users_treatment))
z_score = st.norm.ppf(1 - confidence_level / 2)
margin_of_error = se_pooled * z_score

#Calculate dhat, the estimated difference between probability of conversions in the experiment and control groups
d_hat = (conversions_treatment / total_users_treatment) - (conversions_control / total_users_control)

#Test if we can reject the null hypothesis
lower_bound = d_hat - margin_of_error
upper_bound = d_hat + margin_of_error

if practical_significance < lower_bound:
    print("Se rechaza la hipótesis nula")
else: 
    print("No se rechaza la hipótesis nula")
    
print("La cota inferior del intervalo de confianza es: ", round(lower_bound * 100, 2), "%")
print("La cota superior del intervalo de confianza es: ", round(upper_bound * 100, 2), "%")

No se rechaza la hipótesis nula
La cota inferior del intervalo de confianza es:  -0.38 %
La cota superior del intervalo de confianza es:  0.09 %


Se calcula el error estándar combinado (se_pooled) utilizando la fórmula matemática que tiene en cuenta la probabilidad combinada (prob_pooled), el tamaño de muestra en ambos grupos (total_users_control y total_users_treatment), y la corrección por continuidad en la distribución normal. El resultado se guarda en la variable "se_pooled".

Se calcula el valor z-score (z_score) utilizando la función "norm.ppf" de la biblioteca "scipy.stats" para obtener el valor crítico de la distribución normal estándar correspondiente a un nivel de confianza de (1 - confidence_level / 2). El resultado se guarda en la variable "z_score".

Se calcula el margen de error (margin_of_error) multiplicando el error estándar combinado (se_pooled) por el valor z-score (z_score). El resultado se guarda en la variable "margin_of_error".

Se calcula la estimación de la diferencia entre las probabilidades de conversión en los grupos de tratamiento y control (d_hat) restando la probabilidad de conversión en el grupo de control (conversions_control / total_users_control) de la probabilidad de conversión en el grupo de tratamiento (conversions_treatment / total_users_treatment).

Se calcula el límite inferior del intervalo de confianza (lower_bound) restando el margen de error (margin_of_error) de la estimación de la diferencia (d_hat).

Se calcula el límite superior del intervalo de confianza (upper_bound) sumando el margen de error (margin_of_error) a la estimación de la diferencia (d_hat).

Se realiza una prueba de hipótesis comparando el valor práctico de significancia (practical_significance) con el límite inferior del intervalo de confianza (lower_bound). Si el valor práctico de significancia es menor que el límite inferior del intervalo de confianza, se imprime "Reject null hypothesis", lo cual implica que se puede rechazar la hipótesis nula. En caso contrario, se imprime "Do not reject the null hypothesis", lo cual implica que no se puede rechazar la hipótesis nula.

Se imprime el límite inferior del intervalo de confianza (lower_bound) redondeado a dos decimales como porcentaje.

Se imprime el límite superior del intervalo de confianza (upper_bound) redondeado a dos decimales como porcentaje.